In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
from pyspark import SparkFiles

# Create a SparkSession
spark = SparkSession.builder.appName("TempViews").getOrCreate()

In [3]:
# Read the data from the csv on GitHub
from pyspark import SparkFiles
url = 'https://raw.githubusercontent.com/DKogge/project4/main/Resources/bank-additional-full.csv'
spark.sparkContext.addFile(url)


bank_additional_full = spark.read.option('header', 'true').csv(SparkFiles.get("bank-additional-full.csv"), inferSchema=True, sep=';', timestampFormat="mm/dd/yy")

# Show DataFrame
bank_additional_full.show()


+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|        job| marital|          education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|  housemaid| married|           basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57|   services| married|        high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.

In [4]:
 # Print our schema
bank_additional_full.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp.var.rate: double (nullable = true)
 |-- cons.price.idx: double (nullable = true)
 |-- cons.conf.idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr.employed: double (nullable = true)
 |-- y: string (nullable = true)



In [5]:
# Fill Missing Values
bank_additional_filled = bank_additional_full.na.fill(0)

# Show DataFrame
bank_additional_filled.show()


+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|        job| marital|          education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|  housemaid| married|           basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57|   services| married|        high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.

In [6]:
# Rename the default column to `default credit` and show only this new column
df = bank_additional_filled
df = df.withColumnRenamed('default', 'default credit')
df.show()

+---+-----------+--------+-------------------+--------------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|        job| marital|          education|default credit|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+---+-----------+--------+-------------------+--------------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|  housemaid| married|           basic.4y|            no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57|   services| married|        high.school|       unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|n

In [7]:
# Converting Pyspark DataFRame to Pandas DataFrame
import pandas as pd
pd.set_option('display.max_columns', None)
bank_marketing_df = df.toPandas()

#Show the DataFframe
bank_marketing_df.head()

,age,job,marital,education,default credit,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [8]:
#Save the Dataframe to a csvFile
bank_marketing_df.to_csv('output.csv', index=False)

In [9]:
# Determine the number of unique values in each column.
bank_marketing_df.nunique()

age                 78
job                 12
marital              4
education            8
default credit       3
housing              3
loan                 3
contact              2
month               10
day_of_week          5
duration          1544
campaign            42
pdays               27
previous             8
poutcome             3
emp.var.rate        10
cons.price.idx      26
cons.conf.idx       26
euribor3m          316
nr.employed         11
y                    2
dtype: int64

In [10]:
bank_marketing_df = bank_marketing_df.drop(columns=['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m'])
bank_marketing_df.head()

,age,job,marital,education,default credit,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,5191.0,no


In [11]:
# Check the Age value counts
bank_marketing_df['age'].value_counts()

age
31    1947
32    1846
33    1833
36    1780
35    1759
      ... 
89       2
91       2
94       1
87       1
95       1
Name: count, Length: 78, dtype: int64

In [12]:
# Create bins for the ages
bins = [0, 19, 29, 39, 49, 59, 69, 79, 89, 99]

# Create labels for these bins
group_labels = ["teens", "20s", "30s", "40s", "50s", "60s", "70s", "80s", "90s"]


In [13]:
# Place the data series into a new column inside of the DataFrame
bank_marketing_df["age_group"] = pd.cut(bank_marketing_df["age"], bins, labels=group_labels)
bank_marketing_df.head()

,age,job,marital,education,default credit,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,nr.employed,y,age_group
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,5191.0,no,50s
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,5191.0,no,50s
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,5191.0,no,30s
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,5191.0,no,40s
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,5191.0,no,50s


In [14]:
# drop the age column and rely on the new age bins
bank_marketing_df = bank_marketing_df.drop(columns="age")
bank_marketing_df.head()

,job,marital,education,default credit,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,nr.employed,y,age_group
0,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,5191.0,no,50s
1,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,5191.0,no,50s
2,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,5191.0,no,30s
3,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,5191.0,no,40s
4,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,5191.0,no,50s


In [15]:
bank_marketing_df.dtypes

job                 object
marital             object
education           object
default credit      object
housing             object
loan                object
contact             object
month               object
day_of_week         object
duration             int32
campaign             int32
pdays                int32
previous             int32
poutcome            object
nr.employed        float64
y                   object
age_group         category
dtype: object

In [16]:
bank_marketing_df["age_group"] = bank_marketing_df["age_group"].astype(object)
bank_marketing_df.dtypes

job                object
marital            object
education          object
default credit     object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int32
campaign            int32
pdays               int32
previous            int32
poutcome           object
nr.employed       float64
y                  object
age_group          object
dtype: object

In [17]:
 # Check the Job value counts to identify and replace with "Other"
bank_marketing_df['job'].value_counts()

job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64

In [18]:
# Choose a cutoff value and create a list of jobs to be replaced
# use the variable name `job_to_replace`
value_counts = bank_marketing_df['job'].value_counts()
job_to_replace = value_counts[value_counts < 1050].index

# Replace in dataframe
for jobs in job_to_replace:
  bank_marketing_df['job'] = bank_marketing_df['job'].replace(jobs,"Other")

# Check to make sure replacement was successful
bank_marketing_df['job'].value_counts()

job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
Other             2219
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
Name: count, dtype: int64

In [19]:
# Check the value_counts
bank_marketing_df['marital'].value_counts()

marital
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64

In [20]:
# Check the value_counts
bank_marketing_df['education'].value_counts()

education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: count, dtype: int64

In [21]:
# Generate our categorical variable lists and check the number of unique values in each column
bank_marketing_cat = bank_marketing_df.dtypes[bank_marketing_df.dtypes == "object"].index.tolist()
bank_marketing_df[bank_marketing_cat].nunique()

job               10
marital            4
education          8
default credit     3
housing            3
loan               3
contact            2
month             10
day_of_week        5
poutcome           3
y                  2
age_group          9
dtype: int64

In [22]:
# Convert categorical data to numeric with `pd.get_dummies`
bank_marketing_dummies_df = pd.get_dummies(bank_marketing_df, columns=['job', 'marital', 'education', 'default credit', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'y', 'age_group'], drop_first=True)
bank_marketing_dummies_df = bank_marketing_dummies_df.astype(int)
bank_marketing_dummies_df.head()

,duration,campaign,pdays,previous,nr.employed,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_technician,marital_married,marital_single,marital_unknown,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default credit_unknown,default credit_yes,housing_unknown,housing_yes,loan_unknown,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success,y_yes,age_group_30s,age_group_40s,age_group_50s,age_group_60s,age_group_70s,age_group_80s,age_group_90s,age_group_teens
0,261,1,999,0,5191,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,149,1,999,0,5191,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,226,1,999,0,5191,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0
3,151,1,999,0,5191,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
4,307,1,999,0,5191,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [23]:
#shorten DF name
bank_df = bank_marketing_dummies_df.copy()
bank_df.head()

,duration,campaign,pdays,previous,nr.employed,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_technician,marital_married,marital_single,marital_unknown,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default credit_unknown,default credit_yes,housing_unknown,housing_yes,loan_unknown,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success,y_yes,age_group_30s,age_group_40s,age_group_50s,age_group_60s,age_group_70s,age_group_80s,age_group_90s,age_group_teens
0,261,1,999,0,5191,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,149,1,999,0,5191,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,226,1,999,0,5191,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0
3,151,1,999,0,5191,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
4,307,1,999,0,5191,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [24]:
# Import more dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [25]:
# Split our preprocessed data into our features and target arrays
y = bank_df["y_yes"]
X = bank_df.drop(columns="y_yes")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [26]:
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
# Define the model  *** first iteration to set a baseline ***
number_input_features = (X_train.shape[1])
hidden_nodes_layer1 =  5
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 275       
                                                                 
 dense_1 (Dense)             (None, 5)                 30        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 311 (1.21 KB)
Trainable params: 311 (1.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
966/966 [==============================] - 3s 2ms/step - loss: 0.3527 - accuracy: 0.8498
Epoch 2/100
966/966 [==============================] - 2s 2ms/step - loss: 0.2026 - accuracy: 0.9066
Epoch 3/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1926 - accuracy: 0.9081
Epoch 4/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1894 - accuracy: 0.9087
Epoch 5/100
966/966 [==============================] - 3s 3ms/step - loss: 0.1881 - accuracy: 0.9104
Epoch 6/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1870 - accuracy: 0.9108
Epoch 7/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1863 - accuracy: 0.9101
Epoch 8/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1857 - accuracy: 0.9113
Epoch 9/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1852 - accuracy: 0.9123
Epoch 10/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1849 - accura

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

322/322 - 1s - loss: 0.1842 - accuracy: 0.9133 - 561ms/epoch - 2ms/step
Loss: 0.1841544359922409, Accuracy: 0.9132757186889648


In [31]:
# Export our model to HDF5 file
nn.save("optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
